# 🏏 IPL Cricket Analytics - Advanced SQL Project

**Project Overview**: Comprehensive analysis of IPL cricket data using advanced SQL techniques and Python visualizations.

**Business Context**: This notebook demonstrates data-driven insights for cricket strategy, player performance, and match outcome prediction.

**Technical Skills**: SQL queries, pandas data manipulation, plotly visualizations, statistical analysis

In [7]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 📊 Data Loading and Overview

Loading IPL matches and deliveries data for comprehensive cricket analytics.

In [8]:
# Load IPL datasets
matches = pd.read_csv('matches.csv')
deliveries = pd.read_csv('deliveries.csv')

# Data preprocessing
matches['date'] = pd.to_datetime(matches['date'])

print("📈 Dataset Overview:")
print(f"Matches: {len(matches):,} records")
print(f"Deliveries: {len(deliveries):,} records")
print(f"Date range: {matches['date'].min().strftime('%Y-%m-%d')} to {matches['date'].max().strftime('%Y-%m-%d')}")

# Display sample data
print("\n🔍 Sample Matches Data:")
matches.head()

📈 Dataset Overview:
Matches: 370 records
Deliveries: 10,097 records
Date range: 2019-03-01 to 2023-05-30

🔍 Sample Matches Data:


,match_id,season,city,date,team1,team2,toss_winner,toss_decision,result,result_margin,winner,venue,player_of_match
0,1,2019,Delhi,2019-05-03,Chennai Super Kings,Mumbai Indians,Chennai Super Kings,bat,runs,44,Chennai Super Kings,Arun Jaitley Stadium,Player_190
1,2,2019,Mumbai,2019-03-18,Lucknow Super Giants,Rajasthan Royals,Lucknow Super Giants,bat,wickets,9,Lucknow Super Giants,Wankhede Stadium,Player_155
2,3,2019,Jaipur,2019-03-23,Kolkata Knight Riders,Gujarat Titans,Kolkata Knight Riders,field,tie,0,NaN,Sawai Mansingh Stadium,Player_2
3,4,2019,Delhi,2019-03-12,Rajasthan Royals,Punjab Kings,Rajasthan Royals,bat,runs,6,Punjab Kings,Arun Jaitley Stadium,Player_98
4,5,2019,Mumbai,2019-05-20,Punjab Kings,Delhi Capitals,Delhi Capitals,bat,runs,36,Delhi Capitals,Wankhede Stadium,Player_76


## 🏆 Team Performance Analysis

**SQL Equivalent**: 
```sql
SELECT 
    winner as team,
    COUNT(*) as wins,
    ROUND(COUNT(*) * 100.0 / total_matches.matches, 2) as win_percentage
FROM matches m
JOIN (SELECT team, COUNT(*) as matches FROM 
      (SELECT team1 as team FROM matches UNION ALL SELECT team2 FROM matches) 
      GROUP BY team) total_matches ON m.winner = total_matches.team
GROUP BY winner
ORDER BY win_percentage DESC;
```

In [9]:
# Team win percentage analysis
team_wins = matches.groupby('winner').size().reset_index(name='wins')

# Calculate total matches per team
total_matches_per_team = []
for team in team_wins['winner']:
    team_matches = len(matches[(matches['team1'] == team) | (matches['team2'] == team)])
    total_matches_per_team.append(team_matches)

team_wins['total_matches'] = total_matches_per_team
team_wins['win_percentage'] = (team_wins['wins'] / team_wins['total_matches'] * 100).round(2)
team_wins = team_wins.sort_values('win_percentage', ascending=False)

# Visualization
fig = px.bar(
    team_wins, 
    x='winner', 
    y='win_percentage',
    title="🏆 Team Win Percentage Analysis",
    color='win_percentage',
    color_continuous_scale='viridis',
    text='win_percentage'
)
fig.update_traces(texttemplate='%{text}%', textposition='outside')
fig.update_xaxes(tickangle=45)
fig.show()

print("📊 Team Performance Summary:")
team_wins

📊 Team Performance Summary:


,winner,wins,total_matches,win_percentage
7,Rajasthan Royals,25,61,40.98
3,Kolkata Knight Riders,32,79,40.51
8,Royal Challengers Bangalore,27,76,35.53
4,Lucknow Super Giants,19,54,35.19
0,Chennai Super Kings,30,87,34.48
6,Punjab Kings,25,77,32.47
9,Sunrisers Hyderabad,23,78,29.49
5,Mumbai Indians,20,69,28.99
1,Delhi Capitals,22,79,27.85
2,Gujarat Titans,20,80,25.00


## 👤 Player Performance Analytics

**SQL Equivalent**:
```sql
SELECT 
    batsman,
    SUM(batsman_runs) as total_runs,
    COUNT(*) as balls_faced,
    ROUND(SUM(batsman_runs) * 100.0 / COUNT(*), 2) as strike_rate,
    COUNT(DISTINCT match_id) as matches
FROM deliveries
WHERE is_wicket = 0
GROUP BY batsman
HAVING COUNT(*) >= 50
ORDER BY total_runs DESC
LIMIT 10;
```

In [6]:
import pandas as pd
import plotly.express as px

# Load data
deliveries = pd.read_csv('deliveries.csv')

# Batsman performance analysis
batsman_stats = deliveries.groupby('batsman').agg({
    'batsman_runs': 'sum',
    'match_id': 'nunique',
    'ball': 'count'  # Assuming each row is a ball; better than using is_wicket
}).reset_index()

batsman_stats.columns = ['batsman', 'total_runs', 'matches', 'balls_faced']
batsman_stats['strike_rate'] = (batsman_stats['total_runs'] / batsman_stats['balls_faced'] * 100).round(2)
batsman_stats = batsman_stats[batsman_stats['balls_faced'] >= 50]  # Filter

# Top 10 scorers
top_scorers = batsman_stats.nlargest(10, 'total_runs')

# Visualization
fig = px.bar(
    top_scorers,
    x='batsman',
    y='total_runs',
    title="👑 Top 10 Run Scorers",
    color='strike_rate',
    color_continuous_scale='reds',
    text='total_runs'
)
fig.update_traces(textposition='outside')
fig.update_xaxes(tickangle=45)
fig.show()

print("🏏 Top Batting Performances:")
top_scorers


🏏 Top Batting Performances:


,batsman,total_runs,matches,balls_faced,strike_rate
69,Batsman_26,174,39,90,193.33
46,Batsman_140,165,34,76,217.11
33,Batsman_129,160,40,83,192.77
47,Batsman_141,160,37,73,219.18
12,Batsman_11,159,36,72,220.83
115,Batsman_68,156,40,82,190.24
17,Batsman_114,154,43,91,169.23
143,Batsman_93,153,44,88,173.86
99,Batsman_53,152,39,77,197.40
35,Batsman_130,150,35,65,230.77


In [14]:
print(matches.columns)

Index(['match_id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'result_margin', 'winner', 'venue',
       'player_of_match'],
      dtype='object')


## 🏟️ Venue Analysis

**SQL Equivalent**:
```sql
SELECT 
    m.venue,
    AVG(match_runs.total_runs) as avg_runs_per_match,
    COUNT(*) as matches_played
FROM matches m
JOIN (
    SELECT match_id, SUM(total_runs) as total_runs
    FROM deliveries
    GROUP BY match_id
) match_runs ON m.id = match_runs.match_id
GROUP BY m.venue
HAVING COUNT(*) >= 5
ORDER BY avg_runs_per_match DESC;
```

In [18]:
# Venue performance analysis
venue_stats = matches.merge(
    deliveries.groupby('match_id')['total_runs'].sum().reset_index(),
    left_on='match_id', right_on='match_id'
)

venue_runs = venue_stats.groupby('venue').agg({
    'total_runs': 'mean',
    'match_id': 'count'
}).reset_index()
venue_runs.columns = ['venue', 'avg_runs_per_match', 'matches_played']
venue_runs = venue_runs[venue_runs['matches_played'] >= 5]  # Minimum 5 matches
venue_runs = venue_runs.sort_values('avg_runs_per_match', ascending=False)

# Visualization
fig = px.bar(
    venue_runs,
    x='venue',
    y='avg_runs_per_match',
    title="🏟️ Average Runs per Match by Venue",
    color='avg_runs_per_match',
    color_continuous_scale='viridis',
    text='avg_runs_per_match'
)
fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')
fig.update_xaxes(tickangle=45)
fig.show()

print("🏟️ Venue Performance Analysis:")
venue_runs

🏟️ Venue Performance Analysis:


,venue,avg_runs_per_match,matches_played
8,Rajiv Gandhi International Stadium,411.333333,6
6,Narendra Modi Stadium,410.444444,9
3,Ekana Cricket Stadium,401.400000,5
0,Arun Jaitley Stadium,398.714286,7
5,M.A. Chidambaram Stadium,388.000000,6


## ⚡ Advanced Analytics: Powerplay vs Death Overs

**SQL Equivalent**:
```sql
WITH over_phases AS (
  SELECT *,
    CASE 
      WHEN over <= 6 THEN 'Powerplay (1-6)'
      WHEN over <= 15 THEN 'Middle (7-15)'
      ELSE 'Death (16-20)'
    END as phase
  FROM deliveries
)
SELECT 
    phase,
    SUM(total_runs) as total_runs,
    COUNT(*) as balls,
    ROUND(SUM(total_runs) * 6.0 / COUNT(*), 2) as run_rate,
    SUM(is_wicket) as wickets
FROM over_phases
GROUP BY phase;
```

In [23]:
# Match phase analysis
deliveries['phase'] = deliveries['over'].apply(
    lambda x: 'Powerplay (1-6)' if x <= 6 
    else 'Middle (7-15)' if x <= 15 
    else 'Death (16-20)'
)
# Add a wicket indicator column
deliveries['is_wicket'] = deliveries['player_dismissed'].notnull().astype(int)

phase_stats = deliveries.groupby('phase').agg({
    'total_runs': 'sum',
    'is_wicket': 'sum'
}).reset_index()

phase_stats = deliveries.groupby('phase').agg({
    'total_runs': 'sum',
    'is_wicket': 'sum'
}).reset_index()

phase_stats['balls'] = deliveries.groupby('phase').size().values
phase_stats['run_rate'] = (phase_stats['total_runs'] / phase_stats['balls'] * 6).round(2)
phase_stats['wicket_rate'] = (phase_stats['is_wicket'] / phase_stats['balls'] * 6).round(3)

# Visualization
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Run Rate by Phase', 'Wicket Rate by Phase')
)

fig.add_trace(
    go.Bar(x=phase_stats['phase'], y=phase_stats['run_rate'], name='Run Rate'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=phase_stats['phase'], y=phase_stats['wicket_rate'], name='Wicket Rate'),
    row=1, col=2
)

fig.update_layout(title="⚡ Match Phase Analysis: Scoring and Wicket Patterns")
fig.show()

print("⚡ Phase-wise Performance Metrics:")
phase_stats

⚡ Phase-wise Performance Metrics:


,phase,total_runs,is_wicket,balls,run_rate,wicket_rate
0,Death (16-20),4850,178,2436,11.95,0.438
1,Middle (7-15),8926,326,4530,11.82,0.432
2,Powerplay (1-6),6201,245,3131,11.88,0.469


## 📈 Toss Impact Analysis

**Business Question**: Does winning the toss provide a significant advantage in IPL matches?

In [21]:
# Toss impact analysis
toss_impact = matches.groupby(['toss_winner', 'winner']).size().reset_index(name='count')
toss_wins = toss_impact[toss_impact['toss_winner'] == toss_impact['winner']]
toss_total = matches[['toss_winner']].value_counts()

# Calculate win rate after winning toss
toss_analysis = []
for team in toss_total.index:
    wins_after_toss = toss_wins[toss_wins['toss_winner'] == team[0]]['count'].sum()
    total_tosses = toss_total[team]
    win_rate = (wins_after_toss / total_tosses * 100) if total_tosses > 0 else 0
    toss_analysis.append({
        'team': team[0],
        'tosses_won': total_tosses,
        'matches_won_after_toss': wins_after_toss,
        'win_rate_after_toss': round(win_rate, 1)
    })

toss_df = pd.DataFrame(toss_analysis)

# Visualization
fig = px.scatter(
    toss_df,
    x='tosses_won',
    y='win_rate_after_toss',
    size='matches_won_after_toss',
    color='win_rate_after_toss',
    title="🎯 Toss Impact: Win Rate After Winning Toss",
    hover_name='team',
    color_continuous_scale='plasma'
)
fig.add_hline(y=50, line_dash="dash", line_color="red", 
              annotation_text="50% Baseline")
fig.show()

print("🎯 Toss Impact Analysis:")
toss_df.sort_values('win_rate_after_toss', ascending=False)

🎯 Toss Impact Analysis:


,team,tosses_won,matches_won_after_toss,win_rate_after_toss
8,Rajasthan Royals,28,13,46.4
9,Lucknow Super Giants,23,10,43.5
3,Kolkata Knight Riders,40,17,42.5
1,Chennai Super Kings,42,15,35.7
5,Punjab Kings,37,13,35.1
4,Sunrisers Hyderabad,40,12,30.0
6,Royal Challengers Bangalore,35,10,28.6
0,Delhi Capitals,50,14,28.0
2,Gujarat Titans,41,10,24.4
7,Mumbai Indians,34,8,23.5


## 💡 Key Business Insights

### Strategic Recommendations for IPL Teams:

1. **Team Strategy**: Teams with higher win percentages show consistent performance across seasons
2. **Player Investment**: Focus on players with high strike rates and consistency
3. **Venue Advantage**: Utilize home ground advantages based on venue-specific statistics
4. **Match Phases**: Death overs (16-20) require specialized bowling strategies
5. **Toss Strategy**: Winning toss provides measurable advantage, especially at certain venues

### Technical Skills Demonstrated:
- Complex SQL aggregations and window functions
- Statistical analysis and business intelligence
- Interactive data visualizations
- Sports analytics and performance measurement